In [62]:
import csv
from scipy.constants import h,e,c
import numpy as np
import os

# Photoelectric Effect

In [63]:
def PhotoelectricEffect(stop_voltage:float, nu_0:float=None, work_function_eV:float=None)->float:
    """photoelectric effect

    Args:
        stop_voltage (float): voltage of the circuit
        nu_0 (float): work function frequency (in Hz). Defaults to None.
        work_function_eV (float): Work function (in eV). Defaults to None.

    Returns:
        float: Minimum frequency of the incident beam
    """    
    if nu_0 is not None and work_function_eV is not None:
        raise ValueError("You must give just one of either nu_0 or work_function_eV.")
    elif work_function_eV is None and nu_0 is None:
        raise ValueError("You must give either nu_0 or work_function_eV.")
    elif work_function_eV is not None:
        W = work_function_eV*1.60217733E-19
    else:
        W=h*nu_0
    return (e*stop_voltage+W)/h

## Example

In [64]:
frequency=PhotoelectricEffect(stop_voltage=0.01,work_function_eV=0.5)
print(f"{frequency/1e15:.2f}xe15 Hz")

0.12xe15 Hz


# Materials and voltages available

In [65]:
work_function_materials={'Cu':4.53, 'Ca': 2.87,'Cs': 2.1 ,'Fe': 4.67, 'Co': 5.0, 'Lowest_Work_Function':1}
voltages = [2**(i/50) for i in range(-250,250)]

# Wavelength converter

In [66]:
def toWavelength(frequency:float)->float:
    """to wavelength conversion function

    Args:
        frequency (float): _description_

    Returns:
        float: _description_
    """    
    return c/frequency

In [67]:
wavelength=toWavelength(frequency)
print(f"*Frequency*: {frequency/1e15:.2f}xe15 Hz | *Wavelength*: {wavelength*1e9:.2f} nm")

*Frequency*: 0.12xe15 Hz | *Wavelenght*: 2431.06 nm


# Defining the matrix

# Writing the data into csv file

# Checking the range

In [68]:
from typing import Tuple


def wave2rgb(wave:float)->Tuple[float,float,float,str]:
    # This is a port of javascript code from  http://stackoverflow.com/a/14917481
    gamma = 0.8
    intensity_max = 1

    if wave < 380:
        red, green, blue = -(wave - 440) / (440 - 380), 0, 1
    elif wave < 440:
        red = -(wave - 440) / (440 - 380)
        green, blue = 0, 1
    elif wave < 490:
        red = 0
        green = (wave - 440) / (490 - 440)
        blue = 1
    elif wave < 510:
        red, green = 0, 1
        blue = -(wave - 510) / (510 - 490)
    elif wave < 580:
        red = (wave - 510) / (580 - 510)
        green, blue = 1, 0
    elif wave < 645:
        red = 1
        green = -(wave - 645) / (645 - 580)
        blue = 0
    elif wave <= 780:
        red, green, blue = 1, 0, 0
    else:
        red, green, blue = 0, 0, 0

    # let the intensity fall of near the vision limits
    if wave < 380:
        factor = 0
        return 0.37,0,0.37,"UV"
    elif wave < 420:
        factor = 0.3 + 0.7 * (wave - 380) / (420 - 380)
    elif wave < 700:
        factor = 1
    elif wave <= 780:
        factor = 0.3 + 0.7 * (780 - wave) / (780 - 700)
    else:
        factor = 0
        return 0.37,0,0,"IR"

    def f(c):
        return 0 if c == 0 else intensity_max * pow (c * factor, gamma)

    return f(red), f(green), f(blue), "VIS"

In [69]:
import matplotlib
r,g,b,spec=wave2rgb(wavelength*1e9)
print(f"({r},{g},{b}), {spec})")
print(matplotlib.colors.to_hex([ r, g, b ]))

(0.37,0,0), IR)
#5e0000


In [70]:
fields=['Name','Material','Work Function','Voltage','Frequency','Wavelength',"Color RGB","Color Hex", "Spectrum range"]
val_list = list(work_function_materials.values())
key_list = list(work_function_materials.keys())
matrix1 = []
k=0
for i in work_function_materials.values():
    for j in voltages:
        material_name=key_list[val_list.index(i)]
        voltage=round(j,4)
        frequency=PhotoelectricEffect(stop_voltage=j, work_function_eV=i)
        wavelength=toWavelength(frequency)*1e9
        r=wave2rgb(wavelength)[0]
        g=wave2rgb(wavelength)[1]
        b=wave2rgb(wavelength)[2]
        matrix1.append([f"{material_name},{voltage}",material_name,i, voltage, round(frequency/1e15,4),round(wavelength,4),f"(R={r},G={g},B={b},A=1)",matplotlib.colors.to_hex(wave2rgb(wavelength)[:3]),wave2rgb(wavelength)[3]])
        k=k+1


In [71]:
with open('phe_data.csv', 'w') as file: 
    # creating a csv writer object 
    csv_writer = csv.writer(file) 
        
    # writing the fields 
    csv_writer.writerow(fields) 
    # writing the data rows 
    csv_writer.writerows(matrix1)

In [74]:
fields=['Name','Material','Work Function','Voltage','Frequency','Wavelength',"Color RGB","Color Hex", "Spectrum range"]


for material in work_function_materials.keys():
    matrix2=[]
    for k, j in enumerate(voltages, start=1):
        voltage=round(j,4)
        frequency=PhotoelectricEffect(stop_voltage=j, work_function_eV=work_function_materials[material])
        wavelength=toWavelength(frequency)*1e9
        r=wave2rgb(wavelength)[0]
        g=wave2rgb(wavelength)[1]
        b=wave2rgb(wavelength)[2]
        matrix2.append([k,material,work_function_materials[material], voltage, round(frequency/1e15,4),round(wavelength,4),f"(R={r},G={g},B={b},A=1)",matplotlib.colors.to_hex(wave2rgb(wavelength)[:3]),wave2rgb(wavelength)[3]])
    PATH = os.getcwd()
    data_path = f"{PATH}/data"
    if not os.path.exists(data_path):
            os.mkdir(data_path)
    with open(f"{data_path}/{material}.csv", "w") as file:
        # creating a csv writer object 
        csv_writer = csv.writer(file)  
        # writing the fields 
        csv_writer.writerow(fields) 
        # writing the data rows 
        csv_writer.writerows(matrix2)